# پروژه پایانی

<img src="https://media.geeksforgeeks.org/wp-content/uploads/20200219152327/conv-layers-vgg16.jpg" alt="VGG 16 Architecture" width=500px>
<img src="./pic/vgg16.png" alt="VGG 16 Architecture">


## افزودن کتابخانه‌ها

In [1]:
import numpy as np
import torch
import torch.nn as nn # برای پیاده‌سازی معماری شبکه
from torchvision import datasets # موجود بودن دیتاست مورد نظر در این کتابخانه
from torchvision import transforms # برای کارهای پیش‌پردازش تصویر
from torch.utils.data.sampler import SubsetRandomSampler # نمونه‌برداری تصادفی


# انتخاب نوع پردازنده
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Loading the Data

In [2]:
def data_loader(data_dir, batch_size, random_seed=123, valid_size=0.1, shuffle=True, test=False):

    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010],
    )

    # پیش‌پردازش تصویر
    transform = transforms.Compose([
            transforms.Resize((227,227)),
            transforms.ToTensor(), # تبدیل به تسنور برای استفاده از جی‌پی‌یو
            normalize,
    ])

    if test:
        dataset = datasets.CIFAR100(
          root=data_dir, train=False,
          download=True, transform=transform,
        )

        data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=batch_size, shuffle=shuffle
        )

        return data_loader

    # load the dataset
    train_dataset = datasets.CIFAR100(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler)

    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)

In [3]:
# CIFAR100 دیتاست
train_loader, valid_loader = data_loader(data_dir='./data', batch_size=64)

test_loader = data_loader(data_dir='./data', batch_size=64, test=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## VGG Network

In [4]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2= nn.Sequential(
            nn.Linear(4096, num_classes))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

## Hyperparameters

In [5]:
num_classes = 100
num_epochs = 20
batch_size = 16
learning_rate = 0.005

model = VGG16(num_classes).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.005, momentum=0.9)


# Train the model
total_step = len(train_loader)

## آموزش - Train
> به دلیل حجیم بودن این معماری، نزدیک به چهارده میلیون پارامتر، این که مدل را از صفر آموزش دهیم باعث صرف زمان خیلی خیلی زیادی میشود. به همین دلیل تا اندکی از را اجرا کرده‌ام

In [6]:
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print (f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: [{loss.item():.4f}]")

    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

        print(f"Accuracy of the network on the [{5000}] validation images: [{100 * correct / total} %]")

Epoch [1/20], Step [1/704], Loss: 4.6159
Epoch [1/20], Step [2/704], Loss: 4.6327
Epoch [1/20], Step [3/704], Loss: 4.6542
Epoch [1/20], Step [4/704], Loss: 4.6874
Epoch [1/20], Step [5/704], Loss: 4.6634
Epoch [1/20], Step [6/704], Loss: 4.6515
Epoch [1/20], Step [7/704], Loss: 4.6271
Epoch [1/20], Step [8/704], Loss: 4.6674
Epoch [1/20], Step [9/704], Loss: 4.6177
Epoch [1/20], Step [10/704], Loss: 4.6153
Epoch [1/20], Step [11/704], Loss: 4.6632
Epoch [1/20], Step [12/704], Loss: 4.6545
Epoch [1/20], Step [13/704], Loss: 4.6506
Epoch [1/20], Step [14/704], Loss: 4.6200
Epoch [1/20], Step [15/704], Loss: 4.6399
Epoch [1/20], Step [16/704], Loss: 4.5816
Epoch [1/20], Step [17/704], Loss: 4.5591
Epoch [1/20], Step [18/704], Loss: 4.5742
Epoch [1/20], Step [19/704], Loss: 4.6781
Epoch [1/20], Step [20/704], Loss: 4.6258
Epoch [1/20], Step [21/704], Loss: 4.5796
Epoch [1/20], Step [22/704], Loss: 4.5994
Epoch [1/20], Step [23/704], Loss: 4.7216
Epoch [1/20], Step [24/704], Loss: 4.6748
E

KeyboardInterrupt: 

## آزمایش - Test

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        del images, labels, outputs

    print(f"Accuracy of the network on the [{10000}] test images: [{100 * correct / total   } %]")